In [1]:
import requests
import json
from tqdm import tqdm
import pandas as pd 
import csv
import time
import random

ModuleNotFoundError: No module named 'tqdm'

In [4]:
nha_sach_tiki_url = "https://tiki.vn/api/personalish/v1/blocks/listings?limit=40&include=advertisement&aggregations=2&version=home-persionalized&trackity_id=7588cc77-9f00-4829-803b-dcf326be58e0&category=8322&page={}&urlKey=nha-sach-tiki"
product_url = "https://tiki.vn/api/v2/products/{}"
review_url = "https://tiki.vn/api/v2/reviews?product_id={}"

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36", "Content-Type": "application/json; charset=utf-8"}

In [5]:
def crawl_product_id():
    product_list = []
    i = 1
    while True:
        print("Crawl page: ", i)
        response =  requests.get(nha_sach_tiki_url.format(i), headers=headers)
        if(response.status_code != 200):
            break
        products = json.loads(response.text)["data"]
        if(len(products) == 0):
            break
        for product in products:
            product_id = str(product["id"])
            product_list.append(product_id)
        i += 1
    return product_list, i

In [6]:
product_list, page = crawl_product_id()

Crawl page:  1
Crawl page:  2
Crawl page:  3
Crawl page:  4
Crawl page:  5
Crawl page:  6
Crawl page:  7
Crawl page:  8
Crawl page:  9
Crawl page:  10
Crawl page:  11
Crawl page:  12
Crawl page:  13
Crawl page:  14
Crawl page:  15
Crawl page:  16
Crawl page:  17
Crawl page:  18
Crawl page:  19
Crawl page:  20
Crawl page:  21
Crawl page:  22
Crawl page:  23
Crawl page:  24
Crawl page:  25
Crawl page:  26
Crawl page:  27
Crawl page:  28
Crawl page:  29
Crawl page:  30
Crawl page:  31
Crawl page:  32
Crawl page:  33
Crawl page:  34
Crawl page:  35
Crawl page:  36
Crawl page:  37
Crawl page:  38
Crawl page:  39
Crawl page:  40
Crawl page:  41
Crawl page:  42
Crawl page:  43
Crawl page:  44
Crawl page:  45
Crawl page:  46
Crawl page:  47
Crawl page:  48
Crawl page:  49
Crawl page:  50
Crawl page:  51


In [21]:
def crawl_review(product_list = []):
    reviews_list = []
    for product_id in tqdm(product_list):
        response = requests.get(review_url.format(product_id), headers=headers)
        if response.status_code == 200:
            reviews = response.json().get('data')
            for review in reviews:
                keys = ['id', 'title', 'content', 'thank_count', 'customer_id', 'rating', 'product_id']
                new_review = {key: review[key] for key in keys}
                reviews_list.append(new_review)
        break
        time.sleep(random.randrange(1, 5))
    return reviews_list

In [22]:
reviews_list = crawl_review(product_list)

  0%|          | 0/2000 [00:00<?, ?it/s]


In [9]:
def save_to_csv(reviews_list, path):
    df = pd.DataFrame(reviews_list[:1])
    df.to_csv(path, index=False, encoding='utf-8', mode='w')
    
    for review in reviews_list[1:]:
        df = pd.DataFrame([review])
        df.to_csv(path, index=False, encoding='utf-8', mode='a', header=False)

In [10]:
#save_to_csv(reviews_list, "./data/review.csv")